In [1]:
import base64
import datetime
import os
import re
import urllib.request
from typing import TypedDict

import IPython
import markdown2
import pandas as pd
import pymysql
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from LLM_get_folder import get_local_folder
from PIL import Image

llm = ChatOpenAI(model="gpt-4o", timeout=120, temperature=0)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "auto-post-update"
wp_url = "https://www.forwardpathway.com/wp-json/wp/v2"
wp_post_url = wp_url + "/posts"
wp_media_url = wp_url + "/media"
wp_tag_url = wp_url + "/tags"

user_id = os.environ["wordpress_username"]
# user app password can be created in the user/edit user/application password
user_app_password = os.environ["wordpress_pass"]

credentials = user_id + ":" + user_app_password
token = base64.b64encode(credentials.encode())
header = {"Authorization": "Basic " + token.decode("utf-8")}

In [2]:
def get_keywords_list():
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
        cursorclass=pymysql.cursors.DictCursor,
    )
    cursor = connection.cursor()
    query = """SELECT cname as keyword,concat("https://www.forwardpathway.com/",postid) as url FROM fp_ranking.`colleges`"""
    cursor.execute(query)
    rows = cursor.fetchall()
    keywords_array = []
    for row in rows:
        keywords_array = keywords_array + [row]

    query = """SELECT keyword, url FROM fp_chatGPT.keywords"""
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        keywords_array = keywords_array + [row]

    cursor.close()
    connection.close()

    keywords = pd.DataFrame(keywords_array, columns=["keyword", "url"])
    keywords = keywords.reset_index(drop=True)

    keywords = keywords.reindex(
        keywords["keyword"].str.len().sort_values(ascending=False).index
    ).reset_index(drop=True)

    return keywords


keywords = get_keywords_list()


def get_update_post_ID():
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
        cursorclass=pymysql.cursors.DictCursor,
    )
    cursor = connection.cursor()
    query = """SELECT t3.ID FROM (SELECT t2.ID,t2.post_modified FROM fp_forwardpathway.`wp_mmcp_term_relationships` t1
JOIN fp_forwardpathway.wp_mmcp_posts t2 ON t2.ID=t1.object_id AND t2.post_status="publish"
WHERE t1.`term_taxonomy_id` IN (3,2294,2295,2293,2180,1,1758,35,2350,2351,36)
GROUP BY t2.ID ORDER BY t2.post_modified ASC LIMIT 10) t3
ORDER BY RAND() LIMIT 1"""
    cursor.execute(query)
    row = cursor.fetchone()
    post_ID = int(row["ID"])

    return post_ID


def insert_keyword_url(content):
    soup = BeautifulSoup(content)
    for key, row in keywords.iterrows():
        keyword = row["keyword"]
        url = row["url"]
        new_tag = soup.new_tag("a", href=url)
        new_tag.string = keyword
        void_tags = ["a", "h1", "h2", "h3", "h4"]
        pattern = re.compile(keyword)
        results = soup.find_all(string=pattern)
        for string_element in results:
            parents_set = set([x.name for x in string_element.parents])
            if any([x in parents_set for x in void_tags]):
                continue
            new_element = BeautifulSoup(
                string_element.replace(keyword, str(new_tag), 1), "html.parser"
            )
            string_element = string_element.replace_with(new_element)
            break
    return str(soup)

In [3]:
class GraphState(TypedDict):
    post_ID: int
    URL: str
    raw_html: str
    original_content: str
    text_content: str
    revised_content: str
    revises: list

In [4]:
def get_post_content(state):
    post_ID = state["post_ID"]
    URL = "https://www.forwardpathway.com/" + str(post_ID)
    response = retrieve_post(post_ID)
    raw_html = response.json()["content"]["rendered"]
    soup_content = BeautifulSoup(raw_html, "html.parser")
    # soup.html.unwrap()
    # soup.body.unwrap()
    imgs = soup_content.find_all("img")
    remove_attrs = set(["srcset", "class", "decoding", "height", "sizes", "width"])
    for img in imgs:
        img_attrs = dict(img.attrs)
        for img_attr in img_attrs:
            if img_attr in remove_attrs:
                del img.attrs[img_attr]
    if soup_content is not None:
        elements = soup_content.find_all(
            True,
            class_=[
                "crp_related",
                "topBanner",
                "bottomBanner",
                "wp-block-advgb-summary",
                "yoast-table-of-contents",
                "exclusiveStatement",
                "companyLocation",
                "CommentsAndShare",
                "AI_Summary",
                "AI_QA",
                "btn-group",
            ],
        )
        for element in elements:
            element.decompose()
        elements = soup_content.find_all(
            True,
            id=[
                "crp_related",
            ],
        )
        for element in elements:
            element.decompose()
        elements = soup_content.findAll(["svg", "style", "script", "noscript"])
        for element in elements:
            element.decompose()
    original_content = str(soup_content.find_all(True))
    return {
        "raw_html": raw_html,
        "original_content": original_content,
        "URL": URL,
    }

In [5]:
class revise_single(BaseModel):
    comment: str = Field(description="文章具体修改意见")
    search_query: str = Field(
        description="文章修改所需资料的具体搜索词条，可以用该词条在Google上搜索所需的资料来修改文章"
    )


class revise_output(BaseModel):
    revises: list[revise_single] = Field(
        description="包含文章修改意见和具体搜索词条的数组"
    )


def get_revise_comments(state):
    post_content = state["original_content"]
    revise_comment_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """今天的日期是{today}，下面将给出一篇网站文章，请对该文章的内容提出具体的修改意见，比如使用数据过时、前后逻辑不清晰或者用词不当等你觉得不好的地方。\
                如果文章太短，比如说文章长度少于2000个中文字，请给出增加文章长度的建议，建议增加的内容也请详细给出。\
                    如果遇到某一个表格太长，比如超过五十行数据的请给出精简表格到五十行并引用出处的修改意见，但不多于二十行数据的不要修改，这条修改意见应优先处理放在第一条意见上。修改意见需要与具体的文章内容一一对应具体给出。\
                    同时对应每一条修改意见请给出具体的搜索词条，用该词条可以在Google上搜索相应的资料，搜索词条与修改意见必须一一对应。\
                    请给出3条修改意见，并将每一条修改意见与搜索词条放入对应的输出数组当中""",
            ),
            ("human", "文章内容: {content}"),
        ]
    )
    today = datetime.datetime.now().strftime("%Y年%m月%d日")
    revise_comment_llm = llm.with_structured_output(revise_output)
    revise_comment_chain = revise_comment_prompt | revise_comment_llm
    response = revise_comment_chain.invoke({"today": today, "content": post_content})
    return {"revises": response.revises}

In [6]:
def revise_post(state):
    revises = state["revises"]
    revise_content = state["original_content"]
    revise_n = 0
    for revise in revises:
        if revise_n == 0:
            revise_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        """你是一名专业的网站内容编辑，下面将给出一篇网站文章，并给你一条修改意见以及与该修改意见相关的一些参考资料，请参考修改意见与参考资料对原文章进行修改，尽量保持原文章的长度不减少。\
                        如果更新内容为一张表格中的数据，请对全表格进行更新，且更新内容必须为参考资料中的内容，如果参考资料中不包含关键信息的，请删除原表格中的内容。\
                        修改后的输出结果请输出修改后全文，需要原文的全文修改版。遇到表格要进行表格排版；各级标题也要全文统一格式排版。\
                        尽量保留原文章中的图片。\
                        如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，只添加参考过的资料，参考资料部分要统一排版并编号，编号采用"1."、"2."、"3."格式。\
                        文章原文如果有参考资料部分的，请用新的参考资料覆盖所有的旧参考资料。最终输出结果必须是markdown格式输出，但是不要在文章最前端添加markdown标志。""",
                    ),
                    (
                        "human",
                        "\n\n修改意见:{comment}\n\n参考资料:{docs}\n\n原文章内容: {content}",
                    ),
                ]
            )
        else:
            revise_prompt = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        """你是一名专业的网站内容编辑，下面将给出一篇网站文章，并给你一条修改意见以及与该修改意见相关的一些参考资料，请参考修改意见与参考资料对原文章进行修改，尽量保持原文章的长度不减少。\
                        如果更新内容为一张表格中的数据，请对全表格进行更新，且更新内容必须为参考资料中的内容，如果参考资料中不包含关键信息的，请删除原表格中的内容。\
                        修改后的输出结果请输出修改后全文，需要原文的全文修改版。遇到表格要进行表格排版；各级标题也要全文统一格式排版。\
                        尽量保留原文章中的图片。\
                        如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，只添加参考过的资料，参考资料部分要与原有的参考资料部分统一排版并编号，合并类似的参考资料。\
                        如果资料做了更新，对应的参考资料也进行更新。最终输出结果必须是markdown格式输出，但是不要在文章最前端添加markdown标志。""",
                    ),
                    (
                        "human",
                        "\n\n修改意见:{comment}\n\n参考资料:{docs}\n\n原文章内容: {content}",
                    ),
                ]
            )
        revise_chain = revise_prompt | llm | StrOutputParser()
        comment = revise.comment
        search_query = revise.search_query
        search = GoogleSerperAPIWrapper()
        results = search.results(search_query + " -filetype:pdf -site:forwardpathway.com")
        links = []
        docs = []
        print("--------get search results done!---------------")
        search_result_count = 0
        for x in results["organic"]:
            if search_result_count >= 2:
                break
            loader = WebBaseLoader(
                x["link"], requests_kwargs={"timeout": 15}, continue_on_failure=True
            )
            docs = docs + loader.load()
            search_result_count += 1

        print("--------load docs done!---------------")
        # text_splitter = TokenTextSplitter(
        #    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
        # )
        # chunked_docs = text_splitter.split_documents(docs)
        # vector = FAISS.from_documents(chunked_docs, embeddings)
        # print("--------vectorize docs done!---------------")
        # query_docs = vector.search(search_query, search_type="similarity",k=5)
        revise_content = revise_chain.invoke(
            {"content": revise_content, "comment": comment, "docs": docs}
        )
        # print(revise_content)
        revise_n += 1
        print("-----------{}x revise done!---------------".format(revise_n))
    return {"revised_content": revise_content}

In [7]:
def post_file(file_path):
    media = {
        "file": open(file_path, "rb"),
        "caption": "LLM_auto_post_test_file_" + file_path,
    }
    response = requests.post(wp_media_url, headers=header, files=media)
    return response


def post_post(post_ID, article_body, featured_media_id=0):
    post_data = {
        "content": article_body,
        "featured_media": featured_media_id,
    }
    response = requests.post(
        wp_post_url + "/" + str(post_ID), headers=header, json=post_data
    )
    return response


def retrieve_post(post_ID):
    response = requests.get(wp_post_url + "/" + str(post_ID), headers=header)
    return response

In [8]:
def update_post(state):
    post_ID = state["post_ID"]
    content = state["revised_content"]
    content = markdown2.markdown(content, extras=["tables"])
    content = insert_keyword_url(content)
    # IPython.display.HTML(content)
    post_post(post_ID, content)
    folder = get_local_folder()
    filepath = os.path.join(folder, "post_rewrite.csv")
    df = pd.DataFrame(
        [[post_ID, state["raw_html"], content]],
        columns=["post_ID", "raw_html", "revised"],
    )
    df.to_csv(filepath, mode="a", index=False,header=False)
    return

In [9]:
######################## Build LangGraph ####################################
workflow = StateGraph(GraphState)
workflow.add_node("get_post_content", get_post_content)
workflow.add_node("get_revise_comments", get_revise_comments)
workflow.add_node("revise_post", revise_post)
workflow.add_node("update_post", update_post)

workflow.set_entry_point("get_post_content")
workflow.add_edge("get_post_content", "get_revise_comments")
workflow.add_edge("get_revise_comments", "revise_post")
workflow.add_edge("revise_post", "update_post")
workflow.add_edge("update_post", END)
app = workflow.compile()

In [10]:
app.invoke({"post_ID": get_update_post_ID()})

--------get search results done!---------------
--------load docs done!---------------
-----------1x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
-----------2x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
-----------3x revise done!---------------


{'post_ID': 45946,
 'URL': 'https://www.forwardpathway.com/45946',
 'raw_html': '\n<p><span style="font-weight: 400;">众所周知，种族问题再美国一直是一个关注度很高的话题。美国人对非裔美国人的种族歧视是一个历史带来的问题。不过随着非裔美国人群体为自己的权利发声，美国社会对于多元化的接受度也逐渐变高。但仍然有一些大学以单一的审美文化作为标准要求所有学生，学生甚至会因为不遵守这样的标准而</span><a href="https://www.forwardpathway.com/817"><b>被美国大学开除</b></a><span style="font-weight: 400;">。当然，美国对于非裔美国人的歧视不只局限于大学生，不同年龄段的人都会因为自己的种族问题而受到伤害。</span>涉及种族歧视的问题，学生非常有可能<a href="https://www.forwardpathway.com/3263">被美国大学处分</a>。</p>\n\n\n\n<h2 class="wp-block-heading">要么剪头发，要么就回家</h2>\n\n\n\n<p><span style="font-weight: 400;">2018年的时候，有一名学生六年级的学生Faith Fennidy（以下简称菲斯）。她是非裔美国人，但是在开学的第一天就不能上课。因为学校认为她接发的辫子不符合学校对于发型的规定。菲斯不会辩驳，只能哭着收拾自己的书包。回家后菲斯将此事告诉了父母，她的母亲有点不太理解。但是为了让孩子赶快回到学校，还是花了很多钱帮菲斯修剪头发。没想到当菲斯在再次回到教室的时候，学校认为她的发型仍然不符合规定。学校对于发发型的规定到底是什么呢？</span></p>\n\n\n\n<p><span style="font-weight: 400;">在学校的学生手册上，对发型的规定是：不管男生女生，学生的发型不允许有任何除自己头发以外的物品，比如假发或者发片。过于时髦或者学校认为不合适的发型也是不允许的。黑人的典型发型就不包括在合格的范围内。</span></p>\n\n\n\n<p><span style="font-weight: 400